In [143]:
!pip install ucimlrepo  --quiet
import matplotlib.pyplot as plt
from ucimlrepo import fetch_ucirepo

In [144]:
predict_students_dropout_and_academic_success = fetch_ucirepo(id=697)

X = predict_students_dropout_and_academic_success.data.features
y = predict_students_dropout_and_academic_success.data.targets

metadata = predict_students_dropout_and_academic_success.metadata

variable_info = predict_students_dropout_and_academic_success.variables
df = X
df['Target'] = y
print(f"{df.shape[0]} entries with {df.shape[1]} features")

4424 entries with 37 features


In [145]:
df.head()

,Marital Status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,Father's qualification,...,Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target
0,1,17,5,171,1,1,122.0,1,19,12,...,0,0,0,0,0.000000,0,10.8,1.4,1.74,Dropout
1,1,15,1,9254,1,1,160.0,1,1,3,...,0,6,6,6,13.666667,0,13.9,-0.3,0.79,Graduate
2,1,1,5,9070,1,1,122.0,1,37,37,...,0,6,0,0,0.000000,0,10.8,1.4,1.74,Dropout
3,1,17,2,9773,1,1,122.0,1,38,37,...,0,6,10,5,12.400000,0,9.4,-0.8,-3.12,Graduate
4,2,39,1,8014,0,1,100.0,1,37,38,...,0,6,6,6,13.000000,0,13.9,-0.3,0.79,Graduate


In [146]:
import re
def generate_variable_map(hard_vars, ds_vars, ds_desc):
    variable_map = {}
    num_vars = len(ds_vars)
    for var_idx in range(num_vars):
        var_name = ds_vars[var_idx]
        if var_name in hard_vars:
            continue
        variable_map[var_name] = {}
        # need to split the description by '-'
        ds_desc[var_idx] = ds_desc[var_idx].replace('–', '-')
        # description = ds_desc[var_idx].split('-')
        description = re.split(r'(?<=\d) - ', ds_desc[var_idx])
        current_key = int(description[0].strip())
        for i, key_value_pair in enumerate(description[1:]):
            # will have the value
            if i < len(description[1:]) - 1:
                current_value = " ".join(key_value_pair.split()[:-1])
                next_key = int(key_value_pair.split()[-1].strip())
            else:
                current_value = key_value_pair.strip()
                next_key = None
            # print(f"({current_key}, {current_value})")
            variable_map[var_name][current_key] = current_value
            current_key = next_key
    return variable_map
predict_students_dropout_and_academic_success.variables['name']
ds_vars = predict_students_dropout_and_academic_success.variables['name']
ds_desc = predict_students_dropout_and_academic_success.variables['description']
# need to create forward mappings for each one of the variables
quantitative_vars = {'Application mode',
'Application order',
'Previous qualification (grade)',
'Admission grade',
'Age at enrollment',
"Curricular units 1st sem (credited)",
"Curricular units 1st sem (enrolled)",
"Curricular units 1st sem (evaluations)",
"Curricular units 1st sem (approved)",
"Curricular units 1st sem (grade)",
"Curricular units 1st sem (without evaluations)",
"Curricular units 2nd sem (credited)",
"Curricular units 2nd sem (enrolled)",
"Curricular units 2nd sem (evaluations)",
"Curricular units 2nd sem (approved)",
"Curricular units 2nd sem (grade)",
"Curricular units 2nd sem (without evaluations)",
"Unemployment rate",
"Inflation rate",
"GDP",
"Target",
}
categorical_vars = {
    'Marital Status',
    'Application mode',
    'Course',
    'Daytime/evening attendance',
    'Previous qualification',
    'Nacionality',
    'Mother\'s qualification',
    'Father\'s qualification',
    'Mother\'s occupation',
    'Father\'s occupation',
    'Displaced',
    'Educational special needs',
    'Debtor',
    'Tuition fees up to date',
    'Gender',
    'Scholarship holder',
    'International'
}

In [147]:
variable_map = generate_variable_map(quantitative_vars, ds_vars, ds_desc)

In [148]:
cat_vars_list = list(categorical_vars)

In [149]:
df.drop(['Inflation rate', 'GDP', "Unemployment rate"], axis=1, inplace=True)

In [150]:
def bin_column(file, num_bins, column, range_min, range_max):
        file[column] = file[column].clip(lower=range_min, upper=range_max)
        bins = pd.cut(file[column], bins=np.linspace(range_min, range_max, num_bins + 1), include_lowest=True, right=True)
        return bins
def normalize_columns(df, columns):
    total = df[columns].sum(axis=1)
    for column in columns:
        df[column] = df[column]/ total
    return df

In [151]:
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import pandas as pd
encoder = OneHotEncoder()

encoded = encoder.fit_transform(df[cat_vars_list])

encoded_df = pd.DataFrame(encoded.toarray(), columns=encoder.get_feature_names_out())

df_one_hot_encoded = pd.concat([df.drop(columns=set(cat_vars_list) - {'Target'}), encoded_df], axis=1)

In [152]:
df_one_hot_encoded["Target"] = df_one_hot_encoded["Target"].replace({'Graduate': 0, 'Dropout': 1})
df_one_hot_encoded

,Application order,Previous qualification (grade),Admission grade,Age at enrollment,Curricular units 1st sem (credited),Curricular units 1st sem (enrolled),Curricular units 1st sem (evaluations),Curricular units 1st sem (approved),Curricular units 1st sem (grade),Curricular units 1st sem (without evaluations),...,Father's occupation_172,Father's occupation_174,Father's occupation_175,Father's occupation_181,Father's occupation_182,Father's occupation_183,Father's occupation_192,Father's occupation_193,Father's occupation_194,Father's occupation_195
0,5,122.0,127.3,20,0,0,0,0,0.000000,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,160.0,142.5,19,0,6,6,6,14.000000,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5,122.0,124.8,19,0,6,0,0,0.000000,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2,122.0,119.6,20,0,6,8,6,13.428571,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,100.0,141.5,45,0,6,9,5,12.333333,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4419,6,125.0,122.2,19,0,6,7,5,13.600000,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4420,2,120.0,119.0,18,0,6,6,6,12.000000,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4421,1,154.0,149.5,30,0,7,8,7,14.912500,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4422,1,180.0,153.8,20,0,5,5,5,13.800000,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [153]:
columns_to_bin = quantitative_vars - categorical_vars - {'Application order', "Unemployment rate", "Inflation rate", "GDP", "Target"}

In [154]:
for column in columns_to_bin:
    print(f"Column '{column}': {df[column].nunique()} unique values")

Column 'Curricular units 1st sem (evaluations)': 35 unique values
Column 'Curricular units 2nd sem (grade)': 782 unique values
Column 'Curricular units 2nd sem (credited)': 19 unique values
Column 'Admission grade': 620 unique values
Column 'Age at enrollment': 46 unique values
Column 'Curricular units 1st sem (enrolled)': 23 unique values
Column 'Curricular units 1st sem (without evaluations)': 11 unique values
Column 'Curricular units 2nd sem (without evaluations)': 10 unique values
Column 'Curricular units 1st sem (credited)': 21 unique values
Column 'Curricular units 1st sem (grade)': 797 unique values
Column 'Curricular units 1st sem (approved)': 23 unique values
Column 'Curricular units 2nd sem (evaluations)': 30 unique values
Column 'Curricular units 2nd sem (approved)': 20 unique values
Column 'Previous qualification (grade)': 101 unique values
Column 'Curricular units 2nd sem (enrolled)': 22 unique values


In [155]:
df_one_hot_encoded_binned = df_one_hot_encoded.copy()
for col in columns_to_bin:
    df_one_hot_encoded_binned[col] = bin_column(df_one_hot_encoded_binned, 5, col, df[col].min(), df[col].max())

In [156]:
df_one_hot_encoded_binned.head()

,Application order,Previous qualification (grade),Admission grade,Age at enrollment,Curricular units 1st sem (credited),Curricular units 1st sem (enrolled),Curricular units 1st sem (evaluations),Curricular units 1st sem (approved),Curricular units 1st sem (grade),Curricular units 1st sem (without evaluations),...,Father's occupation_172,Father's occupation_174,Father's occupation_175,Father's occupation_181,Father's occupation_182,Father's occupation_183,Father's occupation_192,Father's occupation_193,Father's occupation_194,Father's occupation_195
0,5,"(114.0, 133.0]","(114.0, 133.0]","(16.999, 27.6]","(-0.001, 4.0]","(-0.001, 5.2]","(-0.001, 9.0]","(-0.001, 5.2]","(-0.001, 3.775]","(-0.001, 2.4]",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,"(152.0, 171.0]","(133.0, 152.0]","(16.999, 27.6]","(-0.001, 4.0]","(5.2, 10.4]","(-0.001, 9.0]","(5.2, 10.4]","(11.325, 15.1]","(-0.001, 2.4]",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5,"(114.0, 133.0]","(114.0, 133.0]","(16.999, 27.6]","(-0.001, 4.0]","(5.2, 10.4]","(-0.001, 9.0]","(-0.001, 5.2]","(-0.001, 3.775]","(-0.001, 2.4]",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2,"(114.0, 133.0]","(114.0, 133.0]","(16.999, 27.6]","(-0.001, 4.0]","(5.2, 10.4]","(-0.001, 9.0]","(5.2, 10.4]","(11.325, 15.1]","(-0.001, 2.4]",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,"(94.999, 114.0]","(133.0, 152.0]","(38.2, 48.8]","(-0.001, 4.0]","(5.2, 10.4]","(-0.001, 9.0]","(-0.001, 5.2]","(11.325, 15.1]","(-0.001, 2.4]",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [157]:
df_one_hot_encoded_normalized = df_one_hot_encoded.copy()
df_one_hot_encoded_normalized = normalize_columns(df_one_hot_encoded_normalized, list(columns_to_bin))
df_one_hot_encoded_normalized

,Application order,Previous qualification (grade),Admission grade,Age at enrollment,Curricular units 1st sem (credited),Curricular units 1st sem (enrolled),Curricular units 1st sem (evaluations),Curricular units 1st sem (approved),Curricular units 1st sem (grade),Curricular units 1st sem (without evaluations),...,Father's occupation_172,Father's occupation_174,Father's occupation_175,Father's occupation_181,Father's occupation_182,Father's occupation_183,Father's occupation_192,Father's occupation_193,Father's occupation_194,Father's occupation_195
0,5,0.453026,0.472707,0.074267,0.0,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0.415405,0.369970,0.049329,0.0,0.015578,0.015578,0.015578,0.036348,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5,0.439165,0.449244,0.068395,0.0,0.021598,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2,0.371466,0.364158,0.060896,0.0,0.018269,0.024358,0.018269,0.040887,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,0.285850,0.404478,0.128633,0.0,0.017151,0.025727,0.014293,0.035255,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4419,6,0.379401,0.370902,0.057669,0.0,0.018211,0.021246,0.015176,0.041279,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4420,2,0.384615,0.381410,0.057692,0.0,0.019231,0.019231,0.019231,0.038462,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4421,1,0.383168,0.371972,0.074643,0.0,0.017417,0.019905,0.017417,0.037104,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4422,1,0.438383,0.374574,0.048709,0.0,0.012177,0.012177,0.012177,0.033609,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [158]:
df_one_hot_encoded_binned.to_csv("df_one_hot_encoded_binned.csv", index=False)
df_one_hot_encoded_normalized.to_csv("df_one_hot_encoded_normalized.csv", index=False)